# Projet Intermovie

Ce Notebook a pour but d'analyser un dataset de films dans le but de récupérer plusieurs informations :

- La liste des acteurs par film.

- La liste des films Américains (en gardant leur nom en français) et leur note moyenne.

- Les notes moyennes des différents genres.

- La note moyenne de chaque acteur par rapport aux films dans lesquels il apparaît.

## Structure des fichiers tsv

- name.basics :         nconst / primaryName / birthYear / deathYear / primaryProfession / knownForTitles
- title.akas :          titleId / ordering / title / region / language / types / attributes / isOriginalTitle
- title.basics :        tconst / titleType / primaryTitle / originalTitle / isAdult / startYear / endYear / runtimeMinutes / genres
- title.principals :    tconst / ordering / nconst / category / job / characters
- title.ratings :       tconst / averageRating / numVotes

# 1- Liste des acteurs par film

In [ ]:
# Extension IPython rechargeant les modules avant que l'utilisateur saisisse du code.
%load_ext autoreload
%autoreload 2

# Import des librairies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Nettoyage des fichiers tsv 

- Suppression des colonnes inutiles.
- Suppression des lignes comportant des cellules vides.
- Suppression des lignes entièrement identiques.

On réalise cela pour les fichiers name.basics, title.basics et title.principals.

### **Nettoyage du fichier name.basics**

In [ ]:
# On lit le fichier name.basics que l'on insère dans un dataframe.
df_name_basics = pd.read_table("./data/name.basics.tsv")

In [ ]:
# On visualise les noms des colonnes afin de déterminer celles qui doivent être supprimées.
df_name_basics.columns

In [ ]:
# On ne conserve que les colonnes "nconst" et "primaryName".
df_name_basics = df_name_basics.drop(['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], axis = 1)

In [ ]:
# On supprime les lignes comprenant des cellules vides ainsi que les lignes identiques.
df_name_basics = df_name_basics.dropna(axis = 0)
df_name_basics = df_name_basics.drop_duplicates()

In [ ]:
# On vérifie que le nombre total de lignes est bien égal pour l'ensemble des colonnes.
df_name_basics.describe()

In [ ]:
# On exporte ce DataFrame nettoyé au sein d'un fichier CSV.
# df_name_basics.to_csv('name.basics.request_1.csv')

### **Nettoyage du fichier title.basics**

In [ ]:
# On lit le fichier title.basics que l'on insère dans un dataframe.
df_title_basics = pd.read_table("./data/title.basics.tsv")

In [ ]:
# On visualise les noms des colonnes afin de déterminer celles qui doivent être supprimées.
df_title_basics.columns

In [ ]:
# On effectue une regex afin de conserver que les lignes ayant comme 'titleType' -> 'movie'.
# On ne conserve que les colonnes 'tconst', 'titleType' et 'originalTitle'.
df_title_basics = df_title_basics[['tconst', 'titleType', 'originalTitle']][df_title_basics['titleType'].str.contains('movie', regex = True)]

In [ ]:
# On ne conserve que les colonnes 'tconst' et 'originalTitle'.
df_title_basics = df_title_basics.drop(['titleType'], axis = 1)

In [ ]:
# On supprime les lignes comprenant des cellules vides ainsi que les lignes identiques.
df_title_basics = df_title_basics.dropna(axis = 0)
df_title_basics = df_title_basics.drop_duplicates()

In [ ]:
# On vérifie que le nombre total de lignes est bien égal pour l'ensemble des colonnes.
df_title_basics.describe()

In [ ]:
# On exporte ce DataFrame nettoyé au sein d'un fichier CSV.
# df_title_basics.to_csv('title.basics.request_1.csv')

### **Nettoyage du fichier title.principals**

In [ ]:
# On lit le fichier title.principals que l'on insère dans un dataframe.
df_title_principals = pd.read_table("./data/title.principals.tsv")

In [41]:
# On visualise les noms des colonnes afin de déterminer celles qui doivent être supprimées.
df_title_principals.columns

Index(['tconst', 'nconst'], dtype='object')

In [ ]:
# On effectue une regex afin de conserver que les lignes ayant comme 'category' -> 'actor' ou 'actress' ou 'self'.
# On ne conserve que les colonnes 'tconst', 'nconst' et 'category'.
df_title_principals = df_title_principals[['tconst', 'nconst', 'category']][df_title_principals['category'].str.contains('self|actor|actress', regex = True)]

In [ ]:
# On ne conserve que les colonnes 'tconst' et 'nconst'.
df_title_principals = df_title_principals.drop(['category'], axis = 1)

In [ ]:
# On supprime les lignes comprenant des cellules vides ainsi que les lignes identiques.
df_title_principals = df_title_principals.dropna(axis = 0)
df_title_principals = df_title_principals.drop_duplicates()

In [ ]:
# On vérifie que le nombre total de lignes est bien égal pour l'ensemble des colonnes.
df_title_principals.describe()

In [ ]:
# On exporte ce DataFrame nettoyé au sein d'un fichier CSV.
# df_title_principals.to_csv('title.principals.request_1.csv')

## Merge des DataFrames + Nettoyage

On fusionne les DataFrames afin d'avoir dans le même DataFrame les noms des acteurs ainsi que les noms des films. On utilise pour cela nos deux clés d'identification : tconst et nconst.

In [33]:
df_fusion = df_title_principals.merge(df_title_basics, how = 'left', on = 'tconst')

In [36]:
df_fusion = df_fusion.merge(df_name_basics, how = 'left', on = 'nconst')

In [37]:
# On ne conserve que les colonnes 'primaryName' et 'originalTitle'.
df_fusion = df_fusion.drop(['tconst', 'nconst'], axis = 1)

In [38]:
# On supprime les lignes comprenant des cellules vides.
df_fusion = df_fusion.dropna(axis = 0)

In [54]:
df_final = df_fusion.groupby('originalTitle').agg({'primaryName': ','.join}, axis = 0)

In [55]:
# On exporte ce DataFrame correspondant au résultat de la requête dans un fichier CSV.
df_final.to_csv('request_1.csv')